In [1]:
import os
import cv2
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from keras import models
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

dataset_dir = './Dataset'
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')
val_dir = os.path.join(dataset_dir, 'val')

In [2]:
img = cv2.imread(os.path.join(train_dir, 'sofa/2 Seater Sofa.jpg'))

WIDTH,HEIGHT, CHANNELS = img.shape
print(f'the size of the image is ({WIDTH},{HEIGHT},{CHANNELS})')

the size of the image is (80,80,3)


In [3]:
def build_data_pipelines(batch_size, train_data_path, val_data_path, test_data_path):
    
    # Generators to augment data
    train_augmentor = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    train_generator = train_augmentor.flow_from_directory(
        train_dir,
        batch_size=batch_size,
        target_size=(80,80),
        class_mode='categorical'
    )

    val_augmentor = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1. / 255,
    )

    val_generator = val_augmentor.flow_from_directory(
        batch_size=batch_size,
        directory=val_dir,
        target_size=(80,80),
        class_mode='categorical'
    )
    
    test_generator = val_augmentor.flow_from_directory(
        test_data_path,
        class_mode="categorical",
        target_size=(80,80),
        color_mode="rgb",
        shuffle=False,
        batch_size=batch_size
    )
    
    return train_generator, val_generator, test_generator

In [4]:
# model, use transfer learning since small dataset 
def build_model():
    base_model = InceptionV3(weights='imagenet',include_top=False, input_shape=(80,80,3))
    
    model = models.Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    # dont update base model params
    for layer in base_model.layers:
        layer.trainable = False
        
    return model

In [5]:
def train(path_to_data, batch_size=16, epochs=15, learning_rate=1e-4):
    
    dataset_dir = path_to_data
    train_dir = os.path.join(dataset_dir, 'train')
    test_dir = os.path.join(dataset_dir, 'test')
    val_dir = os.path.join(dataset_dir, 'val')

    train_generator, val_generator, test_generator = build_data_pipelines(
        batch_size=batch_size,
        train_data_path=train_dir,
        val_data_path=val_dir,
        test_data_path=test_dir
    )
    
    model = build_model()
    model.summary()
    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    # Train the model
    model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator)

    
    # prediction evaluation
    print("[INFO] Evaluation phase...")

    predictions = model.predict(test_generator)
    # gives a array of the correct class that was predicted based on highest probability
    predictions_idxs = np.argmax(predictions, axis=1)

    # contains metrics about eval phase, gives precision, accuracy , recall, F1-score and support
    # eval_generator.classes-> ground truth
    my_classification_report = classification_report(test_generator.classes, predictions_idxs,
                                                     target_names=test_generator.class_indices.keys())

    # helps to compare predictions of one class to another
    # shows what the model predicted the images to be
    my_confusion_matrix = confusion_matrix(test_generator.classes, predictions_idxs)

    print("[INFO] Classification report :")
    print(my_classification_report)
    print("[INFO] Confusion matrix :")
    print(my_confusion_matrix)
    
    model.save('deployment/saved_model/my_model.h5')

train(path_to_data=dataset_dir,batch_size=15,epochs=20, learning_rate=1e-4)

Found 210 images belonging to 3 classes.
Found 30 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 1, 1, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 22,328,099
Trainable params: 525,315
Non-trainable pa